In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
dataset=torchvision.datasets.ImageFolder("colors",transform=transform)
dataloader=torch.utils.data.DataLoader(dataset, batch_size=3, shuffle=True, num_workers=2)

In [4]:
#get a single item 
x,y=dataset.__getitem__(2)

#get random set of items
dataiter=iter(dataloader)
xs,ys=dataiter.next()

print(ys-1)

tensor([2, 0, 1])


In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model=Model()
loss_fn=nn.CrossEntropyLoss()
optim=torch.optim.SGD(model.parameters(),lr=.1)

In [7]:
for epoch in range(100):
    dataiter=iter(dataloader)
    xs,ys=dataiter.next()
    zs=model(xs)
    
    loss=loss_fn(zs,ys-1)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if epoch % 10 == 0:
        print(loss)

tensor(1.1010, grad_fn=<NllLossBackward>)
tensor(0.8465, grad_fn=<NllLossBackward>)
tensor(0.2819, grad_fn=<NllLossBackward>)
tensor(0.0113, grad_fn=<NllLossBackward>)
tensor(0.0022, grad_fn=<NllLossBackward>)
tensor(0.0012, grad_fn=<NllLossBackward>)
tensor(0.0008, grad_fn=<NllLossBackward>)
tensor(0.0006, grad_fn=<NllLossBackward>)
tensor(0.0005, grad_fn=<NllLossBackward>)
tensor(0.0004, grad_fn=<NllLossBackward>)


In [8]:
dataiter=iter(dataloader)
xs,ys=dataiter.next()
zs=model(xs)
args,indices=zs.max(1)
ys-1,indices

(tensor([2, 0, 1]), tensor([2, 0, 1]))